In [484]:
import math
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot
np.random.seed(1)

In [485]:
def create_place_holders(n_x, n_y):
    if n_x is None or n_y is None:
        return None
    X = tf.placeholder(tf.float32, [n_x, None], "X")
    y = tf.placeholder(tf.float32, [n_y, None], "y")
    return X, y

In [486]:
def initialize_parameters(nx, layers):
    if nx is None or layers is None or len(layers) == 0:
        return None
    parameters = {}
    for i, k in enumerate(layers):
        parameters['W' + str(i+1)] = tf.get_variable('W' + str(i+1), [layers[i], nx], dtype=tf.float32, initializer = tf.contrib.layers.xavier_initializer(seed=1))
        parameters['b' + str(i+1)] = tf.get_variable('b' + str(i+1), [layers[i], 1],  dtype=tf.float32, initializer = tf.zeros_initializer())
        nx = layers[i]
    return parameters
        

In [487]:
def forward_propogation(X, parameters):
    if X is None or parameters is None or len(parameters) == 0:
        return None
    A = X
    for i in range(int(len(parameters)/2)):
        if i == (len(parameters)/2) - 1:
            Z = tf.add(tf.matmul(parameters['W' + str(i+1)], A), parameters['b' + str(i+1)])
            return Z
        Z = tf.add(tf.matmul(parameters['W' + str(i+1)], A), parameters['b' + str(i+1)])
        A = tf.nn.relu(Z) 
    return Z

In [488]:
def compute_cost(Z, Y):
    if Z is None or Y is None:
        return None
    logits = tf.transpose(Z)
    labels = tf.transpose(Y)   
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
    return cost

In [489]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed=0):
    m = X.shape[1]                 
    mini_batches = []
    
    np.random.seed(seed)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    num_complete_minibatches = math.floor(m/mini_batch_size)
    
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [490]:
def model(X_train, Y_train, X_test, Y_test, layers, learning_rate = 0.0001, num_epochs = 100, minibatch_size = 32, print_cost = True):
    if X_train is None or Y_train is None or X_test is None or Y_test is None or layers is None or len(layers) == 0:
        return None
    ops.reset_default_graph()
    (n_x, m) = X_train.shape
    (n_y, m) = Y_train.shape
    tf.set_random_seed(1) 
    seed = 3 
    costs = []
    
    X, Y = create_place_holders(n_x, n_y)
    parameters = initialize_parameters(n_x, layers)
    Z = forward_propogation(X, parameters)
    cost = compute_cost(Z, Y)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    
    num_minibatches = int(m / minibatch_size)
    
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            epoch_cost = 0.0
            seed = seed+1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})                
                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
      # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Z), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
    return parameters
    

In [491]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()
print(X_train_orig)
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T
X_train = X_train_flatten / 255.
X_test = X_test_flatten / 255.
Y_train = convert_to_one_hot(Y_train_orig, 6)
Y_test = convert_to_one_hot(Y_test_orig, 6)

layers =[25, 12, 6]
parameters = model(X_train, Y_train, X_test, Y_test, layers)

[[[[227 220 214]
   [227 221 215]
   [227 222 215]
   ...
   [232 230 224]
   [231 229 222]
   [230 229 221]]

  [[227 221 214]
   [227 221 215]
   [228 221 215]
   ...
   [232 230 224]
   [231 229 222]
   [231 229 221]]

  [[227 221 214]
   [227 221 214]
   [227 221 215]
   ...
   [232 230 224]
   [231 229 223]
   [230 229 221]]

  ...

  [[119  81  51]
   [124  85  55]
   [127  87  58]
   ...
   [210 211 211]
   [211 212 210]
   [210 211 210]]

  [[119  79  51]
   [124  84  55]
   [126  85  56]
   ...
   [210 211 210]
   [210 211 210]
   [209 210 209]]

  [[119  81  51]
   [123  83  55]
   [122  82  54]
   ...
   [209 210 210]
   [209 210 209]
   [208 209 209]]]


 [[[238 232 223]
   [238 232 223]
   [238 232 223]
   ...
   [222 216 209]
   [221 216 207]
   [221 216 206]]

  [[237 232 223]
   [238 232 223]
   [238 232 223]
   ...
   [222 216 209]
   [222 216 208]
   [223 217 207]]

  [[236 232 222]
   [237 232 223]
   [238 232 223]
   ...
   [222 216 209]
   [222 216 208]
   [221 216